## Задание

Данные имеют следующий вид:

    Clothing ID: id одежды
    Age: возраст клиента
    Title: заголовок отзыва по продукту
    Review Text: отзыв по продукту
    Rating: рейтинг продукта от 1 (худшее) до 5 (лучшее)
    Recommended IND: Бинарная переменная указывающая рекомендует ли клиент данный продукт (1) или нет (0)
    Positive Feedback Count: Количество клиентов, нашедших данный продукт хорошим
    Division Name, Department Name, Class Name - категории и подкатегории продукта
    
**_Rating_** - целевая переменная (y), которую требуется предсказывать по имеющимся характеристикам

train.csv - датасет для обучения
<br>
test.csv - датасет для теста, на котором будет производиться вычисление итоговой метрики, результаты предсказанного рейтинга на данном датасате необходимо прислать в csv формате (см. ниже)

Тестируемая метрика: **accuracy**

В качестве решения задания требуется прислать csv файл, а также ноутбук с решением задачи
<br>
Формат файла: Rating (один столбец). Пример см. ниже
<br>
Название файла: Surname_Name_#group.csv (Ivanov_Ivan_316.csv)

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
pd.DataFrame({'Rating': [5]*3})

,Rating
0,5
1,5
2,5


In [5]:
train = pd.read_csv('../data/train_test/train.csv')
train

FileNotFoundError: [Errno 2] File ../data/train_test/train.csv does not exist: '../data/train_test/train.csv'

In [4]:
test = pd.read_csv('../data/train_test/test.csv')
test

,Clothing ID,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1078,31,"Beautiful, but does not fit as pictured",Saw this dress in the blue motif and thought i...,1,0,General,Dresses,Dresses
1,830,28,NaN,I can not imagine the cut in the waist being f...,0,4,General,Tops,Blouses
2,508,43,NaN,NaN,1,0,Initmates,Intimate,Sleep
3,834,54,So chic!,This is a wow! so chic...reminds of something ...,1,2,General Petite,Tops,Blouses
4,651,33,NaN,"This is a very nice bra, upset though because ...",1,0,Initmates,Intimate,Intimates
...,...,...,...,...,...,...,...,...,...
1170,866,26,Quality and style strange,"I liked this top, i really did. i also liked t...",0,2,General,Tops,Knits
1171,868,44,Versatile top,"Bought in the store, will look great with blac...",1,0,General,Tops,Knits
1172,858,31,Hole in it after a month,I have had this shirt for a little over a mont...,0,0,General Petite,Tops,Knits
1173,1082,40,Great for summer to fall,The issues the other reviewers mentioned didn'...,1,0,General,Dresses,Dresses


In [201]:
import spacy #для использования лемматизации
from nltk.corpus import stopwords #для выбрасывания стоп слов
import re
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natalya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [202]:
def preprocess_text(series_column, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    ''' Simple text preprocessing
        
    Parameters:
        series_column - pandas.series
        stop_words - list of stop words
        allowed_postags - what types of words must be kept after lemmatization
        
    Return:
        preprocessed series
    '''
    series_column = series_column.str.lower()
    #series_column = series_column.replace(r'[^a-z]', ' ', regex = True)
    
    for i, text in series_column.iteritems():
        
        # Remove distracting single quotes
        text = re.sub("\'", "", str(text))
        #keep only letters
        text = re.sub('[^a-z]', ' ', str(text))
        
        #drop whitespaces
        text = ' '.join(text.split())
        
        #normalization and drop stop words and too short ones
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if token.pos_ in allowed_postags
                                                   and token.lemma_ not in stop_words
                                                   and len(token.lemma_) > 2]
        
        #old approach but faster than spacy
        #tokens = mystem.lemmatize(text)
        #tokens = [token for token in tokens if token not in [' ', '\n'] 
        #                                      and token not in stop_words
        #                                      and len(token) > 2]
        
        text = ' '.join(tokens)
        series_column[i] = text
    
    return series_column

In [52]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/natalya/anaconda3/lib/python3.8/site-packages/en_core_web_sm -->
/Users/natalya/anaconda3/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [203]:
nlp = spacy.load('en', disable=['parser', 'ner'])
stop_words = stopwords.words('english')

In [204]:
%%time
train['Title'] = preprocess_text(train['Title'], stop_words)
train['Review Text'] = preprocess_text(train['Review Text'], stop_words)


CPU times: user 2min 32s, sys: 1.47 s, total: 2min 33s
Wall time: 2min 39s


In [210]:
train

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,867,47,great top,get top fit great top little dark picture look...,5,1,0,General,Tops,Knits
1,927,54,,poncho pretty unique run sized however style e...,4,1,1,General Petite,Tops,Sweaters
2,1089,34,gorgeous lace,pleased receive dress purchase online color vi...,5,1,3,General,Dresses,Dresses
3,1072,56,summer fall wear,first spot retailer employee pair peasant top ...,5,1,1,General Petite,Dresses,Dresses
4,869,66,new favorite summer blouse,detail blouse lace front sheer look great pers...,5,1,5,General,Tops,Knits
...,...,...,...,...,...,...,...,...,...,...
22306,877,65,,precious tee wash well run large,4,1,1,General,Tops,Knits
22307,828,39,great casual shirt,love shirt casual first time wear receive seve...,4,1,2,General,Tops,Blouses
22308,869,38,,beautiful top fit small cheste wide shoulder n...,3,1,0,General,Tops,Knits
22309,1096,48,miss mark,disappointed run small chest long fabric cheap...,2,0,0,General,Dresses,Dresses


In [206]:
%%time
test['Title'] = preprocess_text(test['Title'], stop_words)
test['Review Text'] = preprocess_text(test['Review Text'], stop_words)

CPU times: user 7.47 s, sys: 71.5 ms, total: 7.54 s
Wall time: 8.04 s


In [207]:
test

,Clothing ID,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1078,31,beautiful fit picture,see dress blue motif think would perfect summe...,1,0,General,Dresses,Dresses
1,830,28,,imagine cut waist flattering order still would...,0,4,General,Tops,Blouses
2,508,43,,,1,0,Initmates,Intimate,Sleep
3,834,54,chic,wow chic remind would buy worth penny even hus...,1,2,General Petite,Tops,Blouses
4,651,33,,nice bra upset though second time order retail...,1,0,Initmates,Intimate,Intimates
...,...,...,...,...,...,...,...,...,...
1170,866,26,quality style strange,like top really also like subtle shimmer fabri...,0,2,General,Tops,Knits
1171,868,44,versatile top,buy store look great black dark blue jean fun ...,1,0,General,Tops,Knits
1172,858,31,hole month,shirt little month wear maybe time small hole ...,0,0,General Petite,Tops,Knits
1173,1082,40,great summer fall,issue reviewer mention bother prior year versi...,1,0,General,Dresses,Dresses


In [211]:
ytrain = train['Rating']
train = train.drop(['Rating'], axis=1)
fulldata = pd.concat([train, test], axis = 0, sort = False)
fulldata

,Clothing ID,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,867,47,great top,get top fit great top little dark picture look...,1,0,General,Tops,Knits
1,927,54,,poncho pretty unique run sized however style e...,1,1,General Petite,Tops,Sweaters
2,1089,34,gorgeous lace,pleased receive dress purchase online color vi...,1,3,General,Dresses,Dresses
3,1072,56,summer fall wear,first spot retailer employee pair peasant top ...,1,1,General Petite,Dresses,Dresses
4,869,66,new favorite summer blouse,detail blouse lace front sheer look great pers...,1,5,General,Tops,Knits
...,...,...,...,...,...,...,...,...,...
1170,866,26,quality style strange,like top really also like subtle shimmer fabri...,0,2,General,Tops,Knits
1171,868,44,versatile top,buy store look great black dark blue jean fun ...,1,0,General,Tops,Knits
1172,858,31,hole month,shirt little month wear maybe time small hole ...,0,0,General Petite,Tops,Knits
1173,1082,40,great summer fall,issue reviewer mention bother prior year versi...,1,0,General,Dresses,Dresses


In [212]:
matrix_text_t = [sent.split() for sent in fulldata['Title']]
matrix_text_r = [sent.split() for sent in fulldata['Review Text']]
#matrix_text_test_t = [sent.split() for sent in test['Title']]
#matrix_text_test_r = [sent.split() for sent in test['Review Text']]

In [213]:
%%time
w2v_t = Word2Vec(matrix_text_t, size=300, sg=0, iter=10, window=5, min_count = 0)
w2v_r = Word2Vec(matrix_text_r, size=300, sg=0, iter=10, window=5, min_count = 0)
#w2v_test_t = Word2Vec(matrix_text_test_t, size=300, sg=0, iter=10, window=5, min_count = 0)
#w2v_test_r = Word2Vec(matrix_text_test_r, size=300, sg=0, iter=10, window=5, min_count = 0)

CPU times: user 23.7 s, sys: 278 ms, total: 24 s
Wall time: 11.1 s


In [214]:
d1 = dict(zip(w2v_t.wv.vocab.keys(), [w2v_t.wv[word] for word in w2v_t.wv.vocab.keys()]))
pd1 = pd.DataFrame.from_dict(d1, orient='index')
d2 = dict(zip(w2v_r.wv.vocab.keys(), [w2v_r.wv[word] for word in w2v_r.wv.vocab.keys()]))
pd2 = pd.DataFrame.from_dict(d2, orient='index')
#d3 = dict(zip(w2v_test_t.wv.vocab.keys(), [w2v_test_t.wv[word] for word in w2v_test_t.wv.vocab.keys()]))
#pd3 = pd.DataFrame.from_dict(d3, orient='index')
#d4 = dict(zip(w2v_test_r.wv.vocab.keys(), [w2v_test_r.wv[word] for word in w2v_test_r.wv.vocab.keys()]))
#pd4 = pd.DataFrame.from_dict(d4, orient='index')

In [115]:
def VectDf(series_column, w2v):
    
    result = np.zeros(series_column.size*300).reshape(series_column.size, 300)
    count = 0
    for i, text in series_column.iteritems():
        word_list = text.split()
        if len(word_list) != 0:
            for wl in word_list:
                #if len(word_list) != 0:
                result[count] += w2v.wv[wl]
        count += 1
        #series_column[i] = result
    return result

In [215]:
df1 = pd.DataFrame(VectDf(train['Title'], w2v_t))
df2 = pd.DataFrame(VectDf(train['Review Text'], w2v_r))
df3 = pd.DataFrame(VectDf(test['Title'], w2v_t))
df4 = pd.DataFrame(VectDf(test['Review Text'], w2v_r))

In [216]:
train = pd.concat([train, df1], axis = 1, sort = False)
train = pd.concat([train, df2], axis = 1, sort = False)
test = pd.concat([test, df3], axis = 1, sort = False)
test = pd.concat([test, df4], axis = 1, sort = False)

In [218]:
test

,Clothing ID,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,0,...,290,291,292,293,294,295,296,297,298,299
0,1078,31,beautiful fit picture,see dress blue motif think would perfect summe...,1,0,General,Dresses,Dresses,0.291236,...,10.551687,1.680396,1.051917,4.669396,-8.991890,-1.510103,-0.117729,-11.593310,3.740277,-11.015422
1,830,28,,imagine cut waist flattering order still would...,0,4,General,Tops,Blouses,0.000000,...,5.620998,-1.383221,-0.774836,0.492647,0.152478,-1.059760,-1.998791,-3.758996,-3.968397,-2.643654
2,508,43,,,1,0,Initmates,Intimate,Sleep,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,834,54,chic,wow chic remind would buy worth penny even hus...,1,2,General Petite,Tops,Blouses,0.087232,...,-3.275778,-2.334515,0.155612,1.160882,-2.142224,0.197243,-0.352219,-5.535091,-1.483355,-2.248438
4,651,33,,nice bra upset though second time order retail...,1,0,Initmates,Intimate,Intimates,0.000000,...,1.136025,0.228799,-3.120135,-0.265690,0.967977,5.547222,-1.059477,-8.685140,-7.267889,-3.456366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,866,26,quality style strange,like top really also like subtle shimmer fabri...,0,2,General,Tops,Knits,0.265835,...,8.806453,-6.705838,1.428497,6.654273,-11.833873,-2.215894,2.558928,-12.181586,-0.826321,-2.630652
1171,868,44,versatile top,buy store look great black dark blue jean fun ...,1,0,General,Tops,Knits,0.163918,...,-2.629718,1.802634,-0.756666,8.222704,-5.746437,-4.542176,12.610598,-12.098671,5.864105,-4.748316
1172,858,31,hole month,shirt little month wear maybe time small hole ...,0,0,General Petite,Tops,Knits,0.082374,...,4.465979,-4.074598,-2.952931,-0.731134,-3.558334,-1.540222,-2.079581,-8.720845,-4.562407,-0.458784
1173,1082,40,great summer fall,issue reviewer mention bother prior year versi...,1,0,General,Dresses,Dresses,0.288991,...,8.478456,-5.796056,-13.380046,-1.077147,-8.206232,-4.467613,1.610205,-9.094642,3.256277,-7.303687


In [143]:
print(train['Division Name'].describe())
print(test['Division Name'].describe())

count       22297
unique          3
top       General
freq        13159
Name: Division Name, dtype: object
count        1175
unique          3
top       General
freq          691
Name: Division Name, dtype: object


In [144]:
print(train['Department Name'].describe())
print(test['Department Name'].describe())

count     22297
unique        6
top        Tops
freq       9928
Name: Department Name, dtype: object
count     1175
unique       6
top       Tops
freq       540
Name: Department Name, dtype: object


In [145]:
print(train['Class Name'].describe())
print(test['Class Name'].describe())

count       22297
unique         20
top       Dresses
freq         5999
Name: Class Name, dtype: object
count        1175
unique         18
top       Dresses
freq          320
Name: Class Name, dtype: object


In [221]:
categorical_columns = ['Division Name', 'Department Name', 'Class Name']

#векторизация небинарных признаков
#train_describe = train.describe(include=[object])
#nonbinary_columns = [c for c in categorical_columns if train_describe[c]['unique'] > 2]
train_nonbinary = pd.get_dummies(train[categorical_columns])

#test_describe = test.describe(include=[object])
#nonbinary_columns_t = [c for c in categorical_columns if test_describe[c]['unique'] > 2]
test_nonbinary = pd.get_dummies(test[categorical_columns])

#присоединяем новые столбцы к датафреймам
train = pd.concat([train, train_nonbinary], axis = 1, sort = False)
test = pd.concat([test, test_nonbinary], axis = 1, sort = False)

#print(len(train_nonbinary.columns))
#print(len(test_nonbinary.columns))

In [222]:
#print(train.shape[0], train.shape[1])
#print(test.shape[0], test.shape[1])
#print(train.columns)
#print(test.columns)
#print(train_nonbinary.columns)
#print(test_nonbinary.columns)

#в тестовой выборке оказалось меньше уникальных значений чем в тренировочной. Отличие именно в этих двух столбцах
test['Class Name_Casual bottoms'] = test.apply(lambda _: 0, axis=1)
test['Class Name_Chemises'] = test.apply(lambda _: 0, axis=1)
test

,Clothing ID,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,0,...,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend,Class Name_Casual bottoms,Class Name_Chemises
0,1078,31,beautiful fit picture,see dress blue motif think would perfect summe...,1,0,General,Dresses,Dresses,0.291236,...,0,0,0,0,0,0,0,0,0,0
1,830,28,,imagine cut waist flattering order still would...,0,4,General,Tops,Blouses,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,508,43,,,1,0,Initmates,Intimate,Sleep,0.000000,...,0,0,0,0,1,0,0,0,0,0
3,834,54,chic,wow chic remind would buy worth penny even hus...,1,2,General Petite,Tops,Blouses,0.087232,...,0,0,0,0,0,0,0,0,0,0
4,651,33,,nice bra upset though second time order retail...,1,0,Initmates,Intimate,Intimates,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,866,26,quality style strange,like top really also like subtle shimmer fabri...,0,2,General,Tops,Knits,0.265835,...,0,0,0,0,0,0,0,0,0,0
1171,868,44,versatile top,buy store look great black dark blue jean fun ...,1,0,General,Tops,Knits,0.163918,...,0,0,0,0,0,0,0,0,0,0
1172,858,31,hole month,shirt little month wear maybe time small hole ...,0,0,General Petite,Tops,Knits,0.082374,...,0,0,0,0,0,0,0,0,0,0
1173,1082,40,great summer fall,issue reviewer mention bother prior year versi...,1,0,General,Dresses,Dresses,0.288991,...,0,0,0,0,0,0,0,0,0,0


In [223]:
print(train.shape[0], train.shape[1])
print(test.shape[0], test.shape[1])

22311 638
1175 638


In [224]:
#удаляем столбцы, для которых уже сделали векторизацию
train = train.drop(['Clothing ID', 'Title', 'Review Text', 'Division Name', 'Department Name', 'Class Name'], axis=1)
test = test.drop(['Clothing ID', 'Title', 'Review Text', 'Division Name', 'Department Name', 'Class Name'], axis=1)
print(train.shape[0], train.shape[1])
print(test.shape[0], test.shape[1])

22311 632
1175 632


In [225]:
r = RandomForestClassifier(max_depth = 25, n_estimators = 400).fit(train, ytrain)
rating = r.predict(test)
rating

array([5, 3, 5, ..., 3, 5, 5])

In [227]:
X_train, X_test, y_train, y_true = train_test_split(train, ytrain, test_size=0.3, shuffle=True)
clf = RandomForestClassifier(max_depth = 25, n_estimators = 400).fit(X_train, y_train)
y_predict = clf.predict(X_test)
accuracy_score(y_true, y_predict)

0.6351956976396773

In [226]:
Rating = pd.DataFrame({'Rating': rating})
Rating.to_csv('TashaYa_rating.csv', index=False)